In [ ]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
!apt update
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,266 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,342 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,950 kB in 2s (1,445 kB/s)
Reading package li

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
def get_well_info(api, well_name):
    # Start the browser and navigate to the search page
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://www.drillingedge.com/search")

    # Locate the search input for well name and API and input the data
    well_name_input = driver.find_element(By.NAME, "well_name")
    api_input = driver.find_element(By.NAME, "api_no")
    well_name_input.send_keys(well_name)
    api_input.send_keys(api)

    search_button = driver.find_element(By.CSS_SELECTOR, "input.btn-submit[value='Search Database']")
    search_button.click()

    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".table.wide-table.interest_table tr")))

    # Find the link within the table cell for well name
    rows = driver.find_elements(By.CSS_SELECTOR, ".table.wide-table.interest_table tr")

    well_link_elem = None
    for row in rows:
        print(row.text)
        try:
            well_link_elem = row.find_element(By.CSS_SELECTOR, f"td a[title*='{well_name}']")
            if well_link_elem:
                break
        except NoSuchElementException:
            continue

    if not well_link_elem:
        print("Well link not found.")
        return  # Exit the function early if no link is found

    main_window_handle = driver.current_window_handle
    # Navigate to the well's page
    well_link_url = well_link_elem.get_attribute('href')
    driver.get(well_link_url)
    print(well_link_url)
    driver.switch_to.window(driver.window_handles[-1])

    def extract_element_text(driver, method, value):
        try:
            if method == "xpath":
                return driver.find_element(By.XPATH, value).text
            elif method == "css":
                return driver.find_element(By.CSS_SELECTOR, value).text
        except NoSuchElementException:
            return "N/A"

    # Extract the required information from the detailed page
    well_status = extract_element_text(driver, "xpath", "//table[contains(@class, 'skinny')]//th[text()='Well Status']/following-sibling::td[1]")
    well_type = extract_element_text(driver, "xpath", "//table[contains(@class, 'skinny')]//th[text()='Well Type']/following-sibling::td[1]")
    closest_city = extract_element_text(driver, "xpath", "//table[contains(@class, 'skinny')]//th[text()='Closest City']/following-sibling::td[1]")
    value_303 = extract_element_text(driver, "css", "p.block_stat span.dropcap:first-of-type")
    value_2k = extract_element_text(driver, "css", "p.block_stat:nth-of-type(2) span.dropcap")

    driver.quit()

    return {
        'well name': well_name,
        'well status': well_status,
        'well type': well_type,
        'closest city': closest_city,
        '303 value': value_303,
        '2k value': value_2k
    }

In [ ]:
csv_file = '/content/drive/Shareddrives/DSCI_560/lab5-part1/FinalData.csv'
df = pd.read_csv(csv_file)
process_data = df[['API#', 'WELL NAME']]
process_data = process_data.dropna(how = 'any')
process_data

,API#,WELL NAME
0,33-053-03846,Dahl Federal 2-15H
1,33-053-06010,Chalmers 5301 44-24 3BR
2,33-053-04981,Colville 5301 44-12T
3,33-053-04069,Magnum 3-36-25H
4,33-053-03944,Magnum 2-36-25H
5,33-053-06012,Chalmers Wade Federal 5301 44-24 12TXR
6,33-053-03936,Jefferies 5301 43-12B
7,33-053-03608,Foley Federal 5301 43-12H
8,33-053-03609,Bray 5301 43-12H
9,33-053-03703,Dahl Federal 15-11H


In [ ]:
# Example usage
results = []

# Iterate over each row in the dataframe
for _, row in process_data.iterrows():
  data = get_well_info(row[0], row[1])
  results.append(data)

filtered_results = [item for item in results if item is not None]
df = pd.DataFrame(filtered_results)
df


API # Well Name Lease Name Location Operator Status
33-053-03846 DAHL FEDERAL 2-15H OASIS PETROLEUM NORTH AMERICA LLC McKenzie County, ND OASIS PETROLEUM NORTH AMERICA LLC Active
https://www.drillingedge.com/north-dakota/mckenzie-county/wells/dahl-federal-2-15h/33-053-03846
API # Well Name Lease Name Location Operator Status
33-053-06010 CHALMERS 5301 44-24 3BR OASIS PETROLEUM NORTH AMERICA LLC McKenzie County, ND OASIS PETROLEUM NORTH AMERICA LLC Active
https://www.drillingedge.com/north-dakota/mckenzie-county/wells/chalmers-5301-44-24-3br/33-053-06010
API # Well Name Lease Name Location Operator Status
33-053-04981 COLVILLE 5301 44-12T OASIS PETROLEUM NORTH AMERICA LLC McKenzie County, ND OASIS PETROLEUM NORTH AMERICA LLC Active
https://www.drillingedge.com/north-dakota/mckenzie-county/wells/colville-5301-44-12t/33-053-04981
API # Well Name Lease Name Location Operator Status
33-053-04069 MAGNUM 3-36-25H SLAWSON EXPLORATION COMPANY, INC. McKenzie County, ND SLAWSON EXPLORATION COMPAN

,well name,well status,well type,closest city,303 value,2k value
0,Dahl Federal 2-15H,Active,Oil & Gas,Williston,118,95
1,Chalmers 5301 44-24 3BR,Active,Oil & Gas,Williston,286,3 k
2,Colville 5301 44-12T,Active,Oil & Gas,Williston,730,2.3 k
3,Magnum 3-36-25H,Active,Oil & Gas,Williston,1.2 k,2.5 k
4,Magnum 2-36-25H,Active,Oil & Gas,Williston,518,627
5,Chalmers Wade Federal 5301 44-24 12TXR,Active,Oil & Gas,Williston,499,589
6,Jefferies 5301 43-12B,Active,Oil & Gas,Williston,1.4 k,1.5 k
7,Foley Federal 5301 43-12H,Active,Oil & Gas,Williston,1.4 k,12.4 k
8,Bray 5301 43-12H,Active,Oil & Gas,Williston,760,3 k
9,Dahl Federal 15-11H,Active,Oil & Gas,Williston,4.2 k,3.4 k


In [ ]:
csv_file_path = '/content/drive/Shareddrives/DSCI_560/lab5-part1/drainedge_data.csv'
df.to_csv(csv_file_path, index=False)

In [ ]:
df1 = pd.read_csv('/content/drive/Shareddrives/DSCI_560/lab5-part1/FinalData.csv')
df2 = pd.read_csv('/content/drive/Shareddrives/DSCI_560/lab5-part1/drainedge_data.csv')
df2 = df2.drop(columns = ['well name'])

combined_df = pd.concat([df1, df2], axis=1)

combined_df.rename(columns = {'303 value': 'value_303', '2k value': 'value_2k', 'API#': 'API_number'}, inplace=True)

combined_df



,File Name,Text Content,Extracted Information,API_number,WELL NAME,WELL NUMBER,LATITUDE,LONGITUDE,ADDRESS,Date,...,Bottom (Ft),Stimulation Stages,Volume,Volume Units,Text Content_info,well status,well type,closest city,value_303,value_2k
0,W21796.pdf,NaN,"{'API#': None, 'WELL NAME': None, 'WELL NUMBER...",33-053-03846,Dahl Federal 2-15H,Dahi Federal 2-15H,48°04'49.25” N,103°39’20.15” W,"1001 Fannin, Suite 1500",NaN,...,NaN,NaN,NaN,NaN,NaN,Active,Oil & Gas,Williston,118,95
1,W28599.pdf,NaN,"{'API#': '33-053-06010', 'WELL NAME': 'Chalmer...",33-053-06010,Chalmers 5301 44-24 3BR,Chalmers 5301 44-24 3BR,NaN,NaN,NaN,10/30/2014,...,20833.0,36.0,206015.0,Barrels,NaN,Active,Oil & Gas,Williston,286,3 k
2,W25571.pdf,NaN,"{'API#': '33-053-04981', 'WELL NAME': 'Colvill...",33-053-04981,Colville 5301 44-12T,24-HOUR PRODUCTION RATE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Active,Oil & Gas,Williston,730,2.3 k
3,W22731.pdf,NaN,"{'API#': '33-053-04069', 'WELL NAME': 'Magnum ...",33-053-04069,Magnum 3-36-25H,24-HOUR PRODUCTION RATE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Active,Oil & Gas,Williston,1.2 k,2.5 k
4,W22249.pdf,NaN,"{'API#': '33-053-03944', 'WELL NAME': None, 'W...",33-053-03944,Magnum 2-36-25H,24-HOUR PRODUCTION RATE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Active,Oil & Gas,Williston,518,627
5,W28601.pdf,NaN,"{'API#': '33-053-06012', 'WELL NAME': 'Chalmer...",33-053-06012,Chalmers Wade Federal 5301 44-24 12TXR,24-HOUR PRODUCTION RATE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Active,Oil & Gas,Williston,499,589
6,W22220.pdf,NaN,"{'API#': '33-053-03911', 'WELL NAME': None, 'W...",33-053-03936,Jefferies 5301 43-12B,24-HOUR PRODUCTION RA TE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Active,Oil & Gas,Williston,1.4 k,1.5 k
7,W20863.pdf,NaN,"{'API#': None, 'WELL NAME': None, 'WELL NUMBER...",33-053-03608,Foley Federal 5301 43-12H,or Facility Name,NaN,NaN,6205 16th Avenue West,NaN,...,NaN,NaN,NaN,NaN,NaN,Active,Oil & Gas,Williston,1.4 k,12.4 k
8,W20864.pdf,NaN,"{'API#': '33-053-03911', 'WELL NAME': None, 'W...",33-053-03609,Bray 5301 43-12H,24-HOUR PRODUCTION RA TE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Active,Oil & Gas,Williston,760,3 k
9,W21266.pdf,NaN,"{'API#': None, 'WELL NAME': None, 'WELL NUMBER...",33-053-03703,Dahl Federal 15-11H,24-HOUR PRODUCTION RATE,NaN,103°38’09.239” W,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Active,Oil & Gas,Williston,4.2 k,3.4 k


In [ ]:
combined_df.to_csv('/content/drive/Shareddrives/DSCI_560/lab5-part1/oil_well_data.csv', index=False)